In [4]:
from lxml import html as lhtml                         # Непосредственно для парсинга.
import requests                                        # Для получения html кода сайта.
from multiprocessing.dummy import Pool as ThreadPool   # Для реализации параллельной обкачки.
from math import ceil                                  # Для округления вверх.
from tqdm.notebook import tqdm                         # Для визуализации прогресса итерации по генератору.
import time                                            # Для ожидания окончания работы в  get_page().
from multiprocessing import Lock                       # Для логгирования.


import numpy as np  # Для дебага, чтобы получать произвольные url'ы.
import random       # Для дебага, чтобы получать произвольные url'ы.

import pandas as pd # Непосредственно для формирования *.csv файла.

In [5]:
with open('authors.txt', 'r') as f:
    authors_id = list(map(int, f.read().split('\n')[:-1]))

In [6]:
way = '//div[@class="rd-listing-product-item rd-listing-product-item_size- rd-listing-product-item_type-"]/a'

In [7]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    r_get = requests.get(url, **kwargs)
    if not r_get.ok:
        time.sleep(t_sleep)
        if n_attempts > 0:
            return get_page(url, n_attempts-1, t_sleep, kwargs)
    return r_get   

In [8]:
def get_url(author):
    
    url_for_download = []

    page_url = "https://www.respublica.ru/authors/" + str(author)
    html = get_page(page_url)
    tree = lhtml.fromstring(html.text)
    all_page_nums = ceil(int(tree.xpath('//span[@class="rd-listing-count__total"]')[0].text) / 22)

    for page_num in range(all_page_nums):
        
        page_url = "https://www.respublica.ru/authors/" + str(author)
        html = get_page(page_url, params={'page': page_num, })
        tree = lhtml.fromstring(html.text)
        all_book_on_page = tree.xpath(way)

        for book in all_book_on_page:
            url_for_download.append("https://www.respublica.ru"+book.attrib['href'])
    return url_for_download

In [9]:
%%time

with ThreadPool(10) as pool:
    authors_book_urls = pool.map(get_url, authors_id)
pool.join()

url_history_for_download = [url for sub_list in authors_book_urls for url in sub_list]

CPU times: user 6.59 s, sys: 161 ms, total: 6.75 s
Wall time: 20.2 s


In [10]:
len(url_history_for_download)

2860

In [12]:
template = {
    "ID":       None,
    "URL":      None,
    "Название": None,
    "Автор":    None,
    "Превью":   None,
    "Изображение":   None,
    "Описание": None,
    "Цена":     None,
    "Цена (старая)": None,
    "В наличии": None,
    "Категория": None,
    "Число отзывов": 0,
    "Число оценок":  0,
    "Оценка":   None,
    "ISBN":     None,
    "Издательство":  None,
    "Серия":    None,
    "Обложка":  None,
    "Формат":   None,
    "Количество страниц":   None,
    "Год издания":   None,
    "Язык":     None,
    "Направление":   None,
    "Страна-производитель": None
}

In [14]:
indexes = np.random.randint(0,len(url_history_for_download), 100)

In [25]:
from multiprocessing import Pool, Lock, Value

mutex = Lock()
n_processed = Value('i', 0)

def get_site(url):
    
    field = template.copy()
    html = get_page(url)
    tree = lhtml.fromstring(html.text)
    
    field["ID"] = tree.xpath('//span[@itemprop="sku"]')[0].text
    field["URL"] = url
    field["Название"] = tree.xpath('//h1[@class="rd-page-product__title"]//text()')[0]
    field["Автор"] = tree.xpath('//a[@itemprop="brand"]//text()')[0]
    
    preview_path = tree.xpath('//a[@class="download-pdf"]')
    if bool(preview_path):
        field["Превью"] = "https://www.respublica.ru" + preview_path[0].attrib['href'] 
    
    field["Изображение"] = tree.xpath('//meta[@property="og:image"]')[0].attrib['content']
    field["Описание"] = ' '.join(tree.xpath('//div[@class="rd-page-product__desc-body"]//text()'))
    field["Цена"] = int(tree.xpath('//span[@class="num"]//text()')[0].replace(' ',''))
    
    old_price_path = tree.xpath('//div[@class="rd-page-product__price-old"]//span//text()')
    if bool(old_price_path):
        field["Цена (старая)"] = old_price_path[0]
    
    field["В наличии"] = bool(tree.xpath('//a[@class="rd-page-product__buy rd-page-product__buy_status_available"]'))
    
    categories = tree.xpath('//div[@class="rd-page-breadcrumbs rd-page-product__breadcrumbs"]//span[@itemprop="name"]//text()')
    field["Категория"] = '; '.join([item[1:] for item in categories])
    
    rating_block = tree.xpath('//span[@itemprop="aggregateRating"]')
    
    if bool(rating_block):
        
        field["Число отзывов"] = int(rating_block[0].xpath('//meta[@itemprop="reviewCount"]')[0].attrib['content'])
        field["Число оценок"] = int(rating_block[0].xpath('//meta[@itemprop="ratingCount"]')[0].attrib['content'])
        field["Оценка"] = float(rating_block[0].xpath('//meta[@itemprop="ratingValue"]')[0].attrib['content'])
    
    features = tree.xpath('//p[@class="rd-page-product__desc-param"]//text()')
    features = {features[i]: features[i+2] for i in range(0,len(features)-1,3)}
    keys = features.keys()
    
    names  = tree.xpath('//p[@class="rd-page-product__desc-param"]//span[@itemprop="name"]')
    values = tree.xpath('//p[@class="rd-page-product__desc-param"]//*[@itemprop="value"]')

    for idx in range(len(names)):
            field[names[idx].text] = values[idx].text
            
    return field

def get_site_wrapper(uid):
    result = get_site(uid)
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return result

In [26]:
%%time

with ThreadPool(100) as pool:
    all_sites = pool.map(get_site_wrapper, url_history_for_download)
pool.join()

2860 objects are processed...CPU times: user 1min 46s, sys: 3.52 s, total: 1min 49s
Wall time: 3min 37s


In [27]:
import pandas as pd
pd.options.display.max_columns=100
sites = pd.DataFrame(all_sites)
sites.head(10)

,ID,URL,Название,Автор,Превью,Изображение,Описание,Цена,Цена (старая),В наличии,Категория,Число отзывов,Число оценок,Оценка,ISBN,Издательство,Серия,Обложка,Формат,Количество страниц,Год издания,Язык,Направление,Страна-производитель,"Вес, г",Раздел,Возрастные ограничения,Жанр,Герои,Эпоха,Возраст,Иллюстратор,Тип,Вид бумаги,Материал,"Размер, см",Пол,Назначение,Тематика,Упаковка,Рисунок,Иллюстрации
0,513925,https://www.respublica.ru/knigi/hudozhestvenna...,Чужак,Стивен Кинг,https://www.respublica.ru/items/373815/downloa...,https://www.respublica.ru/uploads/00/00/00/bz/...,В парке маленького городка Флинт-Сити найден т...,370,460,False,Книги; Художественная литература; Фантастика и...,1,1,5.0,978-5-17-110170-1,АСТ,Темная башня,Твердая,"20 x 12,5",576,2019,Русский,Мистика,Россия,460,Зарубежные,12+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,510392,https://www.respublica.ru/knigi/hudozhestvenna...,На подъеме,Стивен Кинг,https://www.respublica.ru/items/370007/downloa...,https://www.respublica.ru/uploads/01/00/00/bf/...,\r\n\tКасл-Рок — маленький городок в штате Мэн...,290,None,True,Книги; Художественная литература; Фантастика и...,0,0,NaN,978-5-17-112472-4,АСТ,Вселенная Стивена Кинга,Твердая,"16,5 x 12,5",192,2018,Русский,Мистика,Россия,NaN,Зарубежные,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,486522,https://www.respublica.ru/knigi/hudozhestvenna...,Противостояние. 5 июля 1990 - 10 января 1991,Стивен Кинг,https://www.respublica.ru/items/361968/downloa...,https://www.respublica.ru/uploads/00/00/00/6x/...,Америка превратилась в ад. Из секретной лабора...,260,None,True,Книги; Художественная литература; Фантастика и...,0,0,NaN,978-5-17-111821-1,АСТ,Король на все времена(м),Мягкая,"11,5 x 18",992,2018,Русский,Мистика,Россия,468,Зарубежные,16+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,484943,https://www.respublica.ru/knigi/hudozhestvenna...,"Девочка, которая любила Тома Гордона",Стивен Кинг,https://www.respublica.ru/items/360505/downloa...,https://www.respublica.ru/uploads/01/00/00/6o/...,"Девятилетняя Триша заблудилась в лесу, и чем д...",170,None,True,Книги; Художественная литература; Фантастика и...,0,0,NaN,978-5-17-111310-0,АСТ,Современная зарубежная проза,Мягкая,12 х 18,288,2018,Русский,Фэнтези,Россия,150,Зарубежные,0+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,483456,https://www.respublica.ru/knigi/hudozhestvenna...,Мешок с костями,Стивен Кинг,https://www.respublica.ru/items/359228/downloa...,https://www.respublica.ru/uploads/01/00/00/6m/...,"Добро пожаловать в маленький тихий городок, ку...",340,None,True,Книги; Художественная литература; Фантастика и...,0,0,NaN,978-5-17-110698-0,АСТ,Король на все времена,Мягкая,12 х 18,704,2018,Русский,Мистика,Россия,343,Зарубежные,16+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,482933,https://www.respublica.ru/knigi/hudozhestvenna...,После заката,Стивен Кинг,https://www.respublica.ru/items/358652/downloa...,https://www.respublica.ru/uploads/00/00/00/6l/...,"""Команда скелетов"", ""Ночная смена"", ""Все преде...",540,None,True,Книги; Художественная литература; Классическая...,0,0,NaN,978-5-17-111788-7,АСТ,Темная башня,Твердая,"20,5 х 13",416,2018,Русский,None,Россия,NaN,Зарубежные,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,480929,https://www.respublica.ru/knigi/hudozhestvenna...,Секретные окна,Стивен Кинг,https://www.respublica.ru/items/356645/downloa...,https://www.respublica.ru/uploads/01/00/00/6i/...,Вы хотите познакомиться с двумя самыми первыми...,360,None,True,Книги; Художественная литература; Фантастика и...,0,0,NaN,978-5-17-107606-1,АСТ,Вселенная Стивена Кинга,Твердая,13 х 17,544,2018,Русский,Фэнтези,Россия,367,Зарубежные,16+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,471244,https://www.respublica.ru/knigi/hudozhestvenna...,Мешок с костями,Стивен Кинг,https://www.respublica.ru/items/346936/downloa...,https://www.respublica.ru/uploads/01/00/00/61/...,"Доб

In [28]:
sites.shape

(2860, 42)

In [29]:
sites.isnull().sum()

ID                           0
URL                          0
Название                     0
Автор                        0
Превью                    1359
Изображение                  0
Описание                     0
Цена                         0
Цена (старая)             2435
В наличии                    0
Категория                    0
Число отзывов                0
Число оценок                 0
Оценка                    2747
ISBN                       303
Издательство               229
Серия                      777
Обложка                    234
Формат                      19
Количество страниц         236
Год издания                234
Язык                       244
Направление               2099
Страна-производитель       678
Вес, г                    1783
Раздел                     590
Возрастные ограничения    1816
Жанр                      2390
Герои                     2855
Эпоха                     2858
Возраст                   2379
Иллюстратор               2543
Тип     

In [30]:
sites.to_csv('d.harazyan_hw_3.csv')